In [1]:
# 스타벅스 매장정보 수집

# 모듈 불러오기
import selenium
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time



## wait 함수 생성
'''
시도별 매장 수의 차이가 크기 때문에 매장정보 수집을 위한 적절한 timesleep을 선정하는 것이 어려움
매장의 정보를 각각 수집해 시도별로 소요시간을 달리함

wait 함수의 경우 화면에 표시되는 요소에 대한 작업의 대기시간으로 사용 ( 클릭 )
wait2 함수의 경우 화면에 표시되지 않는 요소에 대한 작업의 대기시간으로 사용 (매장정보 수집)

'''
def wait(selector):
    return  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))

def wait2(selector):
    return WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))



# 자료수집

target=f'https://www.starbucks.co.kr/store/store_map.do'

driver=wd.Chrome()
driver.get(target)

## 시도 리스트
driver.find_element(By.CSS_SELECTOR, 'header.loca_search > h3 > a').click() # 지역검색 클릭
sido_opts = driver.find_elements( By.CSS_SELECTOR, 'div.loca_step1_cont > ul > li > a' )
sido_values = [ sido_opt.text.strip() for sido_opt in sido_opts  ] # 세종시 확인을 위한 시도 리스트 생성


## 시도 순회
datas = []          
for i in range(len(sido_values)):  
    wait(f'div.loca_step1_cont > ul > li:nth-child({i+1}) > a')
    driver.find_element(By.CSS_SELECTOR, f'div.loca_step1_cont > ul > li:nth-child({i+1}) > a').click()  # 시도 선택     
## 매장정보 수집, 세종시 별도 처리
    if sido_values[i] != '세종':
        wait(f'#mCSB_2_container li:nth-child(1) > a')
        driver.find_element(By.CSS_SELECTOR, f'#mCSB_2_container li:nth-child(1) > a').click() # 시군구 목록 중 전체 선택
        wait('#mCSB_3_container > ul > li' )
        time.sleep(1.5)     # 매장정보 데이터 로드 시간 확보
        shop_opts = driver.find_elements( By.CSS_SELECTOR, '#mCSB_3_container > ul > li' ) # 시군구(전체)의 매장 갯수 확인
        for j in range(len(shop_opts)): #
            wait2(f"#mCSB_3_container > ul > li:nth-child({j+1})")
            data = driver.execute_script("return arguments[0].textContent;", wait2(f"#mCSB_3_container > ul > li:nth-child({j+1})")) # 각 매장 정보 수집
            datas.append(data.strip())
        wait('#container header.loca_search > h3 > a')
        driver.find_element(By.CSS_SELECTOR, '#container header.loca_search > h3 > a').click() # 지역 검색 클릭으로 시도 목록으로 돌아감
    else:
## 세종시 매장정보 수집 시군구(전체)를 선택하지 않고 시도 선택 후 바로 매장 데이터 수집
        wait('#mCSB_3_container > ul > li' )
        time.sleep(1.5)     # 매장정보 데이터 로드 시간 확보
        shop_opts = driver.find_elements( By.CSS_SELECTOR, '#mCSB_3_container > ul > li' ) # 세종시의 매장 갯수 확인
        for j in range(len(shop_opts)):
            wait2(f"#mCSB_3_container > ul > li:nth-child({j+1})")
            data = driver.execute_script("return arguments[0].textContent;", wait2(f"#mCSB_3_container > ul > li:nth-child({j+1})")) # 각 매장 정보 수집
            datas.append(data.strip())
        wait('#container header.loca_search > h3 > a')
        driver.find_element(By.CSS_SELECTOR, '#container header.loca_search > h3 > a').click() # 지역 검색 클릭으로 시도 목록으로 돌아감
# 드라이버 종료
driver.quit() 

# 수집자료 전처리
## 줄바꿈 기준으로 .split()
datas_s=[]
for i in range(len(datas)):datas_s.append(datas[i].split('\t'))

## 리스트의 원소들을 각각 .strip()
datas_s_s = []
for i in datas_s:
    datas_s_s.append([data.strip() for data in i])

## 시도, 시군구 추출을 위한 주소 세분화
adress=[]
for i in range(len(datas_s_s)):
    adress.append(datas_s_s[i][1].split())

## 타입 정제
for i in range(len(datas_s_s)):
    if datas_s_s[i][0][-1]=='T':
        datas_s_s[i][2]='DT'
    elif datas_s_s[i][0][-1]=='R':
        datas_s_s[i][2]='R'
    else :
        datas_s_s[i][2]='G'

# 제출양식에 맞게 입력
results=[]
for i in range(len(datas)):
    result={
        '지점명':datas_s_s[i][0],
        '주소':datas_s_s[i][1][:-9],
        '시군구':adress[i][1] if adress[i][0]!='세종특별자치시' else '', #세종시 시군구 ''처리
        '시도':adress[i][0],
        '타입': datas_s_s[i][2]  
    }

    
    results.append(result)

# 결과물 출력
results


[{'지점명': '역삼아레나빌딩',
  '주소': '서울특별시 강남구 언주로 425 (역삼동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '논현역사거리',
  '주소': '서울특별시 강남구 강남대로 538 (논현동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '신사역성일빌딩',
  '주소': '서울특별시 강남구 강남대로 584 (논현동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '국기원사거리',
  '주소': '서울특별시 강남구 테헤란로 125 (역삼동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '대치재경빌딩',
  '주소': '서울특별시 강남구 남부순환로 2947 (대치동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '봉은사역',
  '주소': '서울특별시 강남구 봉은사로 619 (삼성동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '압구정윤성빌딩',
  '주소': '서울특별시 강남구 논현로 834 (신사동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '코엑스별마당',
  '주소': '서울특별시 강남구 영동대로 513 (삼성동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'G'},
 {'지점명': '삼성역섬유센터R',
  '주소': '서울특별시 강남구 테헤란로 518 (대치동)',
  '시군구': '강남구',
  '시도': '서울특별시',
  '타입': 'R'},
 {'지점명': '압구정R',
  '주소': '서울특별시 강남구 언주로 861 (신사동)',
  '시군구': '강남구',
  '시도': '서울

In [2]:
len(results)

1904